In [1]:
import pybullet as p
import pybullet_data
import time
import math
import numpy as np
from pinocchio_template import thneed

# Connect to the physics server
physicsClient = p.connect(p.GUI)
p.resetDebugVisualizerCamera(cameraDistance=2.0, cameraYaw=0, cameraPitch=-30, cameraTargetPosition=[0,0,0])

pybullet build time: Jan 29 2025 23:16:28


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3080/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.183.01
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.183.01
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3080/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


ven = NVIDIA Corporation
ven = NVIDIA Corporation


In [2]:
friction_thing_N = 0
disable_velocity_control = False

# Set up the environment
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)
planeId = p.loadURDF("/home/a2rlab/Documents/bullet3/data/plane.urdf")

# Load the URDF model
# Replace "your_model.urdf" with the path to your URDF file
# If using a model from pybullet_data, you can use its name directly
# robotId = p.loadURDF("/home/a2rlab/Documents/bullet3/examples/pybullet/gym/pybullet_data/franka_panda/panda.urdf", [0, 0, 0], useFixedBase=1)
robotId = p.loadURDF("/home/a2rlab/Documents/emre/indy-ros2/indy_description/urdf_files/indy7_nolimits.urdf", [0, 0, 0], useFixedBase=1, flags=p.URDF_USE_INERTIA_FROM_FILE)

num_joints = p.getNumJoints(robotId)
print(f"Number of joints: {num_joints}")

# Print joint info
for i in range(num_joints):
    joint_info = p.getJointInfo(robotId, i)
    print(f"Joint {i}, Name: {joint_info[1].decode('utf-8')}, Type: {joint_info[2]}")

if disable_velocity_control:
    p.setJointMotorControlArray(bodyIndex=robotId,
                            jointIndices=[i for i in range(num_joints)],
                            controlMode=p.VELOCITY_CONTROL,
                            forces=friction_thing_N*np.ones(num_joints))

sim_timestep = 0.01
solve_timestep = 0.01
p.setTimeStep(sim_timestep) # Default is 1/240 seconds

# max_torque = 1000
t = thneed(dt=solve_timestep)
t.max_qp_iters = 5
t.R_cost = 0
t.dQ_cost = 0
xs = 0.2 * np.hstack([np.ones(6), np.zeros(6)])
for i in range(1,7):
    p.resetJointState(robotId, i, xs[i-1], 0)
# goal = np.array([-2.30313706e-01, -1.23076760e-01,  6.17299293e-02])
# goal = np.array([ 1.51468905e-10, -1.86500000e-01,  1.26750000e+00]) # p sure this is 0
goal = t.eepos(0.2 * np.ones(6))
goal_tile = np.tile(goal, t.N).T

all_xs = []
all_ctrl = []

o = 0
fails = 0
for i in range(300):
    all_xs.append(np.array(xs))
    # print(np.linalg.norm(t.eepos(xs[:6]) - goal))
    
    # t.clean_start()
    # t.shift_start()
    # t.XU[:t.nx] = xs

    t.XU = np.zeros(t.traj_len)
    for i in range(t.N):
        t.XU[i*t.nxu:i*t.nxu + t.nx] = xs

    if t.sqp(xs, goal_tile):
        fails +=1
        if fails==t.N-1:
            raise Exception("ran out of traj to track")
    else:
        fails = 0
    
    trqs = t.XU[o*t.nxu+t.nx:o*t.nxu+t.nxu]
    all_ctrl.append(trqs)

    o = 1
    print(t.XU[o*t.nxu:o*t.nxu+t.nq])
    p.setJointMotorControlArray(bodyIndex=robotId,
                                jointIndices=[i for i in range(1,7)],
                                controlMode=p.POSITION_CONTROL,
                                targetPositions=t.XU[o*t.nxu:o*t.nxu+t.nq],
                                # targetVelocities=t.XU[o*t.nxu+t.nq:o*t.nxu+t.nx],
                                forces=[1e5 for _ in range(6)]
                                )
    # p.setJointMotorControlArray(bodyIndex=robotId,
    #                         jointIndices=[i for i in range(1,7)],
    #                         controlMode=p.TORQUE_CONTROL,
    #                         forces=trqs)
    
    
    # Step the simulation forward
    for i in range(int(solve_timestep / sim_timestep)):
        p.stepSimulation()
        time.sleep(sim_timestep)
    

    joint_states = p.getJointStates(robotId, [i for i in range(1,7)])
    q = [j[0] for j in joint_states]
    v = [j[1] for j in joint_states]
    torques_applied = [j[3] for j in joint_states] # this is zeroed for torque control

    # q_rk4, v_rk4 = t.rk4(xs[:t.nq], xs[t.nq:t.nx], trqs, t.dt)
    # print(q - q_rk4, v - v_rk4)

    xs[:t.nq] = q
    xs[t.nq:t.nx] = v

    # print(xs - XU[t.nxu:t.nxu+t.nx])

# Disconnect from the physics server
p.disconnect()

b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: tcp
Number of joints: 8
Joint 0, Name: global, Type: 4
Joint 1, Name: joint0, Type: 0
Joint 2, Name: joint1, Type: 0
Joint 3, Name: joint2, Type: 0
Joint 4, Name: joint3, Type: 0
Joint 5, Name: joint4, Type: 0
Joint 6, Name: joint5, Type: 0
Joint 7, Name: tcp, Type: 4
osqp warm starting: False


: 

In [ ]:
all_xs = np.array(all_xs)
all_ctrl = np.array(all_ctrl)

In [ ]:
print(np.max(all_xs, axis=0) - np.min(all_xs, axis=0))
print(np.max(all_ctrl, axis=0) - np.min(all_ctrl, axis=0))

[0.20000139 0.20000363 0.19999881 0.20001936 0.20000196 0.20002465 1.93187561 1.98856008 1.75301632 1.95357223 1.90464582 1.93466422]
[5.40232931e+01 1.40227215e+02 1.00048993e+02 5.20932166e+01 4.89738044e+01 1.11987241e-01]


In [ ]:
print(sum(t.stats['linesearch_alphas']['values']) / len(t.stats['linesearch_alphas']['values']))
len([a for a in t.stats['linesearch_alphas']['values'] if a==0.0])

0.0038932030728122914


897

In [ ]:
all_ctrl

array([[-7.31161161e+01,  4.90501498e+01,  1.62344348e+02, -6.63958049e+01,  5.84967972e+01, -1.67951990e-01],
       [-1.90928230e+01,  1.19839469e+02,  6.22953544e+01, -1.43025883e+01,  9.52299283e+00, -5.67391137e-02],
       [-1.99523946e+01,  1.16357223e+02,  6.23337545e+01, -1.50379961e+01,  1.02247291e+01, -5.59647483e-02],
       ...,
       [-1.95679512e+01,  1.20859509e+02,  6.30345330e+01, -1.46166194e+01,  9.67673787e+00, -5.88781750e-02],
       [-2.92903079e+01,  1.89277083e+02,  9.96494862e+01, -2.31377131e+01,  1.60361078e+01, -8.11227942e-02],
       [-1.95679523e+01,  1.20859518e+02,  6.30345374e+01, -1.46166201e+01,  9.67673837e+00, -5.88781773e-02]])